# Computación a Gran Escala

**Curso 2022/2023**

## Lab 6: Spark SQL - Tarea 3
**Autores:**  
  
Miguel García González - miguel.garcia02@estudiante.uam.es  
Belén Vivas García - belen.vivas@estudiante.uam.es

### Inicialización de Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder.appName('Lab6-T3').getOrCreate()
sc = spark.sparkContext

/home/belen/spark/venv/spark/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


### Lectura de ficheros

In [3]:
# Rutas de datos
BOOK_RATINGS_PATH = 'data/BX-CSV-Dump/BX-Book-Ratings.csv'
BOOKS_PATH = 'data/BX-CSV-Dump/BX-Books.csv'
USERS_PATH = 'data/BX-CSV-Dump/BX-Users.csv'

In [4]:
book_ratings_df = spark.read.csv(BOOK_RATINGS_PATH, sep=";", header=True)
book_ratings_df = book_ratings_df.withColumn("Book-Rating", book_ratings_df["Book-Rating"].cast('int'))

books_df = spark.read.csv(BOOKS_PATH, sep=";", header=True)
books_df = books_df.withColumn("Year-Of-Publication", books_df["Year-Of-Publication"].cast('int'))

users_df = spark.read.csv(USERS_PATH, sep=";", header=True)
users_df = users_df.withColumn("Age", users_df["Age"].cast('int'))

### 1. Consultas mediante funciones de la API

#### a. Lista de usuarios junto con el número de libros que han valorado

Agrupamos las ratings por usuarios y contamos cuántas tiene cada uno.

In [5]:
book_ratings_df.groupBy('User-Id').count().show()

+-------+-----+
|User-Id|count|
+-------+-----+
| 277594|    1|
| 277840|    1|
| 278220|    2|
| 278659|    1|
|   1436|   12|
|   2136|   10|
|   3959|    2|
|   4032|    2|
|   4821|    3|
|   4937|    5|
|   5325|    1|
|   5925|    1|
|   6613|    1|
|   6731|   26|
|   7711|    1|
|   8433|    1|
|   9030|    1|
|   9583|    1|
|   9586|    2|
|   9993|    1|
+-------+-----+
only showing top 20 rows



#### b. Rating máximo recibido por cada editorial

Primero unimos las tablas de libros y ratings por el ISBN para asociar las ratings de los libros con las editoriales, agrupamos la tabla resultante por editorial y buscamos el rating más alto mediante agregación.

In [6]:
books_df.join(book_ratings_df, on='ISBN').groupBy('Publisher').agg({"Book-Rating": "max"}).show()

+--------------------+----------------+
|           Publisher|max(Book-Rating)|
+--------------------+----------------+
|Harper Mass Marke...|              10|
|Houghton Mifflin ...|              10|
|Carroll &amp; Gra...|              10|
|Orion Business Books|               1|
|        Lorenz Books|              10|
|        Chosen Books|              10|
| Research Press (IL)|               7|
|      Celestial Arts|              10|
|        Book Pub. Co|               0|
|         Hermetic Pr|              10|
|      Aqua Explorers|               9|
|Faith Publishing ...|               8|
|           LPC Group|               9|
|Chicago Review Press|              10|
|    Adams Media Corp|              10|
|         Cleis Press|              10|
|Ullstein-Taschenb...|              10|
|         Bertelsmann|               9|
|Editiones B, Grup...|               0|
|    Macmillan Pub Co|              10|
+--------------------+----------------+
only showing top 20 rows



#### c. Nombre del autor que ha recibido más ratings

Primero unimos las tablas de libros y ratings por el ISBN para asociar las ratings de los libros con los autores, agrupamos la tabla resultante por autor y contamos las ratings. A continuación, pasamos los valores de la columna "count" con las ratings a entero, para poder trabajar con ella, y la ordenamos de forma descendente para tener los valores más altos arriba, quedándonos con el primer valor, es decir, el más alto.

In [7]:
num_ratings_df = books_df.join(book_ratings_df, on='ISBN').groupBy('Book-Author').count()

num_ratings_df = num_ratings_df.withColumn("count", num_ratings_df["count"].cast('int'))

num_ratings_df.orderBy(num_ratings_df['count'].desc()).limit(1).show()

+------------+-----+
| Book-Author|count|
+------------+-----+
|Stephen King|10053|
+------------+-----+



### 2. Consultas mediante Window Functions

#### a. ¿Cuál es el título del libro con mayor número de ratings para cada editorial?

Primero juntamos las tablas de libros y ratings, y agrupamos la tabla resultante por ISBN, título y editorial, contando el número de ratings. Luego, especificamos la función de ventana para hacer el ranking de cada libro según la columna "count" con el número de ratings. Finalmente, sólo mostramos las filas cuyo ranking es 1, porque queremos únicamente el libro con mayor ratings.

In [8]:
ratings_df = books_df.join(book_ratings_df, on='ISBN').groupBy('ISBN', 'Book-Title', 'Publisher').count()

# Hacemos el ranking para cada libro
window = Window.partitionBy('Publisher').orderBy(ratings_df['count'].desc())

ratings_df = ratings_df.withColumn(
    'Ranking', f.rank().over(window)).select(
    'Publisher', 'Book-Title', 'count', 'Ranking')
ratings_df.where('Ranking = 1').show()

+--------------------+--------------------+-----+-------+
|           Publisher|          Book-Title|count|Ranking|
+--------------------+--------------------+-----+-------+
|            3D Press|Denver Hiking Gui...|    1|      1|
|3H Productions, Inc.|Haley's Cleaning ...|    1|      1|
|A &amp; B Book Di...|Heal Thyself for ...|    1|      1|
|A &amp; B Book Di...|The Lost Books of...|    1|      1|
|  A H M Publications|The Subjection of...|    1|      1|
|      A New Hope Pub|     Pent Up Passion|    1|      1|
|        A Plume Book|The Prodigal Proj...|    1|      1|
| A. &amp; M. Muchnik|Comuna Verdad (An...|    1|      1|
|  A. Bonniers förlag|Svalan, katten, r...|    1|      1|
|          A. Deutsch|          Moon Tiger|    2|      1|
|               A.T.E|El triángulo mort...|    1|      1|
|          AAAI Press|Computers and Tho...|    1|      1|
|                  AM|La tahona de Froilán|    1|      1|
|                AVON|       Wedding Night|    2|      1|
|        Abc-C

#### b. ¿Cuál es la diferencia entre el número de ratings de cada libro y el número de ratings del libro con mayor número de ratings de la misma editorial?

Creamos la columna con la diferencia, restando la rating más alta sobre la ventana especificada en la consulta anterior y las ratings. En este caso mostramos más filas para que se aprecie bien el resultado de la consulta.

In [9]:
ratings_df.withColumn('Ratings-Diff', f.max(ratings_df['count']).over(window) - ratings_df['count']).show(500)

+--------------------+--------------------+-----+-------+------------+
|           Publisher|          Book-Title|count|Ranking|Ratings-Diff|
+--------------------+--------------------+-----+-------+------------+
|            3D Press|Denver Hiking Gui...|    1|      1|           0|
|3H Productions, Inc.|Haley's Cleaning ...|    1|      1|           0|
|A &amp; B Book Di...|Heal Thyself for ...|    1|      1|           0|
|A &amp; B Book Di...|The Lost Books of...|    1|      1|           0|
|  A H M Publications|The Subjection of...|    1|      1|           0|
|      A New Hope Pub|     Pent Up Passion|    1|      1|           0|
|        A Plume Book|The Prodigal Proj...|    1|      1|           0|
| A. &amp; M. Muchnik|Comuna Verdad (An...|    1|      1|           0|
|  A. Bonniers förlag|Svalan, katten, r...|    1|      1|           0|
|          A. Deutsch|          Moon Tiger|    2|      1|           0|
|          A. Deutsch|     Apple of my eye|    1|      2|           1|
|     